# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
KEYWORDS = ['python', 'парсинг']
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')
titles_block = list(map(lambda x: x.find('h2', class_='post__title'), posts))
links = list(map(lambda x: x.find('a').get('href'), titles_block)) 

In [3]:
def get_all_links(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    titles_block = list(map(lambda x: x.find('h2', class_='post__title'), posts))
    links = list(map(lambda x: x.find('a').get('href'), titles_block)) 
    return links

all_links = get_all_links('https://habr.com/ru/all/')

In [4]:
def get_right_news(links, keywords):
    all_right_articles = pd.DataFrame()
    for link in links:
        req = requests.get(link).text
        soup = BeautifulSoup(req, 'html.parser')
        time.sleep(0.3)
        article = soup.find('div', class_='post__body').text.lower()
        if any([word in article for word in keywords]):
            date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
            title = soup.find('span', class_='post__title-text').text
            
            res = {'date': date, 'title': title, 'link': link, 'text': article}
            all_right_articles = pd.concat([all_right_articles, pd.DataFrame([res])])
    return all_right_articles

all_articles = get_right_news(all_links, KEYWORDS)

In [5]:
all_articles

,date,title,link,text
0,2020-12-05,Машинное обучение: ансамбль смешивания на Python,https://habr.com/ru/company/skillfactory/blog/...,\nсмешивание — это ансамблевый алгоритм машинн...
0,2020-12-05,Гипертимезия и эволюция в IT,https://habr.com/ru/post/531438/,\nнедавно открыл для себя термин “гипертимезия...


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [8]:
URL_base = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['alpotehinj@gmail.com', 'delluiza@yandex.ru']

In [9]:
def get_list_req(URL, emails):
    list_req = []
    headers = {
        'Vaar-Version': '0',
        'Vaar-Header-App-Product': 'hackcheck-web-avast'  
    }
    for email in emails:
        data = {
            'emailAddresses': [email]
        }
        req = requests.post(URL, json=data, headers=headers)
        print(req.text)
        list_req.append(req.json())
        time.sleep(2)
    return list_req

list_requests = get_list_req(URL_base, EMAIL)

{"breaches":{"3858":{"breachId":3858,"site":"streamcraft.net","recordsCount":2971087,"description":"At an unconfirmed date, StreamCraft's user database was allegedly breached. The stolen data contains over 183,000 records including email addresses and passwords. The compromised data is being shared privately on the darknet.","publishDate":"2017-04-14T00:00:00Z","statistics":{"usernames":2788530,"passwords":2015623,"emails":1964476}},"17830":{"breachId":17830,"site":"wattpad.com","recordsCount":273322456,"description":"In June 2020, the online writing community Wattpad was allegedly breached. The stolen data contains usernames, IPs, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2020-07-23T00:00:00Z","statistics":{"usernames":270846321,"passwords":192540751,"emails":273004868}},"18150":{"breachId":18150,"site":"gonitro.com","recordsCount":86440975,"description":"In October 2020, the PDF editor Nitro 

In [45]:
def get_data_breaches(dict_breaches, emails):
    all_breaches_info = pd.DataFrame()
    for i in range(len(dict_breaches)):
        breach = dict_breaches[i]['breaches']
        email = emails[i]
        for key, value in breach.items():
            date = pd.to_datetime(value['publishDate']).date()
            source = value['site']
            desc = value['description']
            row = {'email': email, 'publish_date': date, 'source_breach': source, 'description': desc}
            all_breaches_info = pd.concat([all_breaches_info, pd.DataFrame([row])])
    return all_breaches_info

all_breaches_info = get_data_breaches(list_requests, EMAIL)

In [46]:
all_breaches_info

,email,publish_date,source_breach,description
0,alpotehinj@gmail.com,2017-04-14,streamcraft.net,"At an unconfirmed date, StreamCraft's user dat..."
0,alpotehinj@gmail.com,2020-07-23,wattpad.com,"In June 2020, the online writing community Wat..."
0,alpotehinj@gmail.com,2020-11-19,gonitro.com,"In October 2020, the PDF editor Nitro was alle..."
0,alpotehinj@gmail.com,2020-11-05,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."
0,delluiza@yandex.ru,2017-04-14,streamcraft.net,"At an unconfirmed date, StreamCraft's user dat..."
0,delluiza@yandex.ru,2020-05-28,stalker.so,"In January 2020, the Russian multiplayer video..."
0,delluiza@yandex.ru,2019-08-08,borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
0,delluiza@yandex.ru,2020-07-23,wattpad.com,"In June 2020, the online writing community Wat..."
0,delluiza@yandex.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,delluiza@yandex.ru,2020-12-03,mindset24global.com,"In November 2020, a collection of over 23,000 ..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
